Contract details
-------------------

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=107)

<IB connected to 127.0.0.1:7497 clientId=107>

Suppose we want to find the contract details for AMD stock.
Let's create a stock object and request the details for it:

In [3]:
amd = Contract(secType="OPT", symbol="DE000KA33LU0")
amd = Contract(conId=382147169)
amd = Contract(secType="IOPT", localSymbol="KA33LU")


cds = ib.reqContractDetails(amd)

len(cds)

2

We get a long list of contract details. Lets print the first one:

In [4]:
cds[0]

ContractDetails(marketName='DAX', minTick=0.001, orderTypes='ACTIVETIM,ADJUST,ALERT,ALLOC,AVGCOST,BASKET,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,LIT,LMT,MIT,MKT,MTL,NONALGO,OCA,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='FWB,SWB', priceMagnifier=1, underConId=825711, longName='DAX 30 Index (Deutsche Aktien Xchange 30)', contractMonth='NOEXP', timeZoneId='MET', tradingHours='20190928:CLOSED;20190929:CLOSED;20190930:0900-20190930:2200;20191001:0900-20191001:2200;20191002:0900-20191002:2200;20191003:CLOSED;20191004:0900-20191004:2200;20191005:CLOSED;20191006:CLOSED;20191007:0900-20191007:2200;20191008:0900-20191008:2200;20191009:0900-20191009:2200;20191010:0900-20191010:2200;20191011:0900-20191011:2200;20191012:CLOSED;20191013:CLOSED;20191014:0900-20191014:2200;20191015:0900-20191015:2200;20191016:0900-20191016:2200;20191017:0900-20191017:2200;20191018:0900-20191018:2200;20191019:CLOSED;20191020:CLOSED;

The contract itself is in the 'summary' property of the contract details. Lets make a list of contracts and look at the first:

In [5]:
contracts = [cd.summary for cd in cds]

contracts

[Contract(secType='IOPT', conId=382147169, symbol='DAX', lastTradeDateOrContractMonth='NOEXP', strike=12081.2263, right='C', multiplier='0.01', exchange='FWB', currency='EUR', localSymbol='KA33LU', tradingClass='DAX'),
 Contract(secType='IOPT', conId=382147169, symbol='DAX', lastTradeDateOrContractMonth='NOEXP', strike=12081.2263, right='C', multiplier='0.01', exchange='SWB', currency='EUR', localSymbol='KA33LU', tradingClass='DAX')]

To better spot the difference between all the contracts it's handy to convert to a DataFrame. There is a utility function to do that:

In [6]:
util.df(contracts)

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,IOPT,382147169,DAX,NOEXP,12081.2263,C,0.01,FWB,,EUR,KA33LU,DAX,False,,,,None,None
1,IOPT,382147169,DAX,NOEXP,12081.2263,C,0.01,SWB,,EUR,KA33LU,DAX,False,,,,None,None


Or use a dict comprehension of set comprehensions...

In [7]:
{key: set(getattr(c, key) for c in contracts) for key in Contract.defaults}

{'secType': {'IOPT'},
 'conId': {382147169},
 'symbol': {'DAX'},
 'lastTradeDateOrContractMonth': {'NOEXP'},
 'strike': {12081.2263},
 'right': {'C'},
 'multiplier': {'0.01'},
 'exchange': {'FWB', 'SWB'},
 'primaryExchange': {''},
 'currency': {'EUR'},
 'localSymbol': {'KA33LU'},
 'tradingClass': {'DAX'},
 'includeExpired': {False},
 'secIdType': {''},
 'secId': {''},
 'comboLegsDescrip': {''},
 'comboLegs': {None},
 'deltaNeutralContract': {None}}

We can see from this that AMD trades in different currencies on different exchanges.
Suppose we want the one in USD on the SMART exchange. The AMD contract is adjusted to
reflect that and becomes unique:

In [8]:
amd = Stock('AMD', 'SMART', 'USD')

assert len(ib.reqContractDetails(amd)) == 1

Lets try the same for Intel:

In [9]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

Let's try a non-existing contract:

In [10]:
xxx = Stock('XXX', 'SMART', 'USD')

assert len(ib.reqContractDetails(xxx)) == 0

Error 200, reqId 6: No security definition has been found for the request, contract: Stock(symbol='XXX', exchange='SMART', currency='USD')


or a Forex contract

In [11]:
eurusd = Forex('EURUSD')

assert len(ib.reqContractDetails(eurusd)) == 1

With the ``qualifyContracts`` method the extra information that is send back
from the contract details request is used to fill in the original contracts.

Lets do that with ``amd`` and compare before and aftwards:

In [12]:
amd

Stock(symbol='AMD', exchange='SMART', currency='USD')

In [13]:
ib.qualifyContracts(amd)
amd

Stock(conId=4391, symbol='AMD', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMD', tradingClass='NMS')

**TIP:** When printing a contract, the output can be copy-pasted and it will be valid Python code.

The ``conId`` that is returned can by itself be used to uniquely specify a contract:

In [14]:
contract_4391 = Contract(conId=4391)

ib.qualifyContracts(contract_4391)

assert contract_4391 == amd

A whole bunch of contracts can be qualified at the same time. A list of all the successfull ones is returned:

In [15]:
qualContracts = ib.qualifyContracts(amd, intc, xxx, eurusd)

assert intc in qualContracts
assert xxx not in qualContracts

Error 200, reqId 12: No security definition has been found for the request, contract: Stock(symbol='XXX', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='XXX', exchange='SMART', currency='USD')


There is also a new API function to request contracts that match a pattern:

In [16]:
matches = ib.reqMatchingSymbols('Intel')
matchContracts = [m.contract for m in matches]

matchContracts

[Contract(secType='STK', conId=172096099, symbol='INTELLECT', primaryExchange='NSE', currency='INR'),
 Contract(secType='STK', conId=12178234, symbol='INTC', primaryExchange='EBS', currency='CHF'),
 Contract(secType='STK', conId=12177196, symbol='INL', primaryExchange='FWB', currency='EUR'),
 Contract(secType='STK', conId=8988636, symbol='4335', primaryExchange='SEHK', currency='HKD'),
 Contract(secType='STK', conId=270639, symbol='INTC', primaryExchange='NASDAQ.NMS', currency='USD'),
 Contract(secType='STK', conId=38709539, symbol='INTC', primaryExchange='MEXI', currency='MXN'),
 Contract(secType='STK', conId=11463493, symbol='INTCUSD', primaryExchange='EBS', currency='USD'),
 Contract(secType='STK', conId=257312042, symbol='002600', primaryExchange='SEHKSZSE', currency='CNH'),
 Contract(secType='STK', conId=165301734, symbol='601877', primaryExchange='SEHKNTL', currency='CNH'),
 Contract(secType='STK', conId=257311331, symbol='002050', primaryExchange='SEHKSZSE', currency='CNH'),
 Co

In [ ]:
assert intc in matchContracts